In [ ]:
from model_GW_dataGen import *
os.chdir('TOVs_Thread/')

### BNS with TEOBResumS

In [ ]:
#%matplotlib inline
from multiprocessing.pool import ThreadPool

#choose name of file where to store final dataset:
file_name = "GW_data.txt"

#create new file - NB: it OVERWRITE if a file with same name exists 
f = open(file_name, "w")
f.close()

#Load speed of sound dataset
#These are encoded in the parameter I called 'z', which is read from the name of file
#Note: these are on separate files than TOV! They will be added to each line at the end
cs_values = readMatrix('matrixcs10000.dat')
rho_values = readMatrix('matrixrho10000.dat')

#Select all needed eos files, right now only with Lambda=0
listFile = []
#First it gets ap4 files
for file in glob.glob('TOV_ap4_0.0*.csv'):
    listFile.append(file)
#Then it gets sly files
for file in glob.glob('TOV_sly_0.0*.csv'):
    listFile.append(file)

#For a test: this is a list with just two files to run the loop and check the code
#listFile = ['TOV_ap4_0.0_4462.csv','TOV_ap4_0.0_3202.csv']

#Threaded loop on EOS:
with ThreadPool() as pool:
    for result in pool.map(task_EOS_loop,listFile):
        #Add info on c_s and rho at beginning of each line (given z, which is removed)
        new_result = []
        for row in result:
            z_val = int(row[0])
            array_to_save = row[1:]
            array_to_save = np.insert(array_to_save, 0, cs_values[z_val-1], axis=0)
            array_to_save = np.insert(array_to_save, 0, rho_values[z_val-1], axis=0)
            new_result.append(array_to_save)
        
        #Dump data into common file
        f = open(file_name, 'a')
        np.savetxt(f, new_result, fmt='%.5e', delimiter=',')
        f.close()